In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier


df_dados = pd.read_csv(r"C:\Users\Igor\Documents\GitHub\DATATHON_2026\DATATHON_2026\data\dados_processados\BASE_DE_DADOS_PEDE_2024_DATATHON_LIMPO.csv", sep=';')

# #Variavel de risco
df_dados['Risco'] = (df_dados['Defasagem'] < 0).astype(int)



features = ['IDA','IEG','IPS','IPP','IAA','IPV','Idade']
X = df_dados[features]
y = df_dados['Risco']


# Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

############################### modelo de classificação ###############################


# Treinar modelo
model_random = RandomForestClassifier(
    n_estimators=300,
    max_depth=10,           
    min_samples_leaf=5,     
    max_features='sqrt',    
    random_state=42,
    n_jobs=-1               
)
model_random.fit(X_train, y_train)


model_xg = XGBClassifier()
model_xg.fit(X_train, y_train)

model_gradient = GradientBoostingClassifier()
model_gradient.fit(X_train, y_train)


# --- Avaliação Random Forest ---
pred_rf = model_random.predict(X_test)
print("### RELATÓRIO RANDOM FOREST ###")
print(classification_report(y_test, pred_rf))

# --- Avaliação XGBoost ---
pred_xg = model_xg.predict(X_test)
print("\n### RELATÓRIO XGBOOST ###")
print(classification_report(y_test, pred_xg))

# --- Avaliação Gradient Boosting ---
pred_gb = model_gradient.predict(X_test)
print("\n### RELATÓRIO GRADIENT BOOSTING ###")
print(classification_report(y_test, pred_gb))

### RELATÓRIO RANDOM FOREST ###
              precision    recall  f1-score   support

           0       0.73      0.71      0.72       196
           1       0.77      0.78      0.78       241

    accuracy                           0.75       437
   macro avg       0.75      0.75      0.75       437
weighted avg       0.75      0.75      0.75       437


### RELATÓRIO XGBOOST ###
              precision    recall  f1-score   support

           0       0.72      0.71      0.71       196
           1       0.77      0.78      0.77       241

    accuracy                           0.75       437
   macro avg       0.74      0.74      0.74       437
weighted avg       0.75      0.75      0.75       437


### RELATÓRIO GRADIENT BOOSTING ###
              precision    recall  f1-score   support

           0       0.71      0.72      0.72       196
           1       0.77      0.76      0.77       241

    accuracy                           0.74       437
   macro avg       0.74      0.7